# **R-Squared Disentangled**

### Import Libraries

In [3]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

### Import Data

In [5]:
dummy = pd.read_excel('C:/Users/limam/Documents/GitHub/Datathon_2023/Code/Python/result_PCA_indicador_dia_diff.xlsx')
dummy

,Data,daily_index,Sentiment Score,PCA_Normalized
0,2023-06-30,56,0.057568,57.099997
1,2023-06-29,54,-0.080815,54.900000
2,2023-06-28,62,0.003656,63.699998
3,2023-06-27,59,0.059167,60.399997
4,2023-06-26,55,0.011786,55.999998
...,...,...,...,...
1960,2018-02-05,11,-0.204696,7.600003
1961,2018-02-04,24,0.134817,21.899995
1962,2018-02-03,40,-0.101355,39.500000
1963,2018-02-02,15,0.078715,11.999997


## *R-Squared*

In [6]:
fg = dummy.drop(['daily_index', 'Sentiment Score'], axis=1).set_index('Data').sort_values(by='Data', ascending=True)
fg['weekly_mean'] = fg['PCA_Normalized'].rolling(7).mean()
fg['sentiment'] = 0

fg.loc[(fg['weekly_mean'] <= 24), 'sentiment'] = 'Extreme Fear'
fg.loc[(fg['weekly_mean'] >24) & (fg['weekly_mean'] <= 44), 'sentiment'] = 'Fear'
fg.loc[(fg['weekly_mean'] > 44) & (fg['weekly_mean'] <= 55), 'sentiment'] = 'Neutral'
fg.loc[(fg['weekly_mean'] > 55) & (fg['weekly_mean'] <= 75), 'sentiment'] = 'Greed'
fg.loc[(fg['weekly_mean'] > 75), 'sentiment'] = 'Extreme Greed'

# fg.loc[(fg['weekly_mean'] <= 44), 'sentiment'] = 'Fear'
# fg.loc[(fg['weekly_mean'] > 44) & (fg['weekly_mean'] <= 55), 'sentiment'] = 'Neutral'
# fg.loc[fg['weekly_mean'] > 55, 'sentiment'] = 'Greed'

# fg.loc[(fg['weekly_mean'] <= 50), 'sentiment'] = 'Fear'
# fg.loc[(fg['weekly_mean'] > 50), 'sentiment'] = 'Greed'

# fg.loc[(fg['weekly_mean'] > 24) & (fg['weekly_mean'] <= 75), 'sentiment'] = 'Neutral'
# fg.loc[(fg['weekly_mean'] <= 24) | (fg['weekly_mean'] > 75), 'sentiment'] = 'Extreme'

fg

,PCA_Normalized,weekly_mean,sentiment
Data,,,
2018-02-01,28.499998,NaN,0
2018-02-02,11.999997,NaN,0
2018-02-03,39.500000,NaN,0
2018-02-04,21.899995,NaN,0
2018-02-05,7.600003,NaN,0
...,...,...,...
2023-06-26,55.999998,61.342855,Greed
2023-06-27,60.399997,62.914283,Greed
2023-06-28,63.699998,63.385712,Greed


In [7]:
tech = pd.read_excel('C:/Users/limam/Documents/GitHub/Datathon_2023/Dados/tech_signals_w.xlsx')
tech['Date'] = pd.to_datetime(tech['Date'])
tech

,Date,Adj Close,Volume,MA_1_9,MA_1_12,MA_2_9,MA_2_12,MA_3_9,MA_3_12,MOM_9,MOM_12,VOL_1_9,VOL_1_12,VOL_2_9,VOL_2_12,VOL_3_9,VOL_3_12
0,2015-03-02,274.354004,242425100,1,1,1,1,1,0,1,0,1,1,1,1,1,1
1,2015-03-09,286.393005,259502000,1,1,1,1,1,1,1,0,1,1,1,1,1,1
2,2015-03-16,267.959991,206778200,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3,2015-03-23,242.712997,179497700,0,0,1,1,1,1,1,0,1,1,1,1,1,1
4,2015-03-30,260.597992,150120700,1,1,0,1,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,2023-10-02,27935.089844,82183775698,1,1,1,1,1,0,0,0,0,0,0,0,0,0
449,2023-10-09,27159.652344,72673654487,1,0,1,1,1,1,1,0,0,0,0,0,0,0
450,2023-10-16,29993.896484,113402383094,1,1,1,1,1,1,1,1,0,0,0,0,0,0
451,2023-10-23,33086.234375,38363572311,1,1,1,1,1,1,1,1,1,0,1,0,0,0


In [8]:
df = tech.merge(fg, how='left', left_on='Date', right_on='Data')
df.dropna(inplace=True)
df.drop('PCA_Normalized', axis=1, inplace=True)
df

,Date,Adj Close,Volume,MA_1_9,MA_1_12,MA_2_9,MA_2_12,MA_3_9,MA_3_12,MOM_9,MOM_12,VOL_1_9,VOL_1_12,VOL_2_9,VOL_2_12,VOL_3_9,VOL_3_12,weekly_mean,sentiment
154,2018-02-12,10551.799805,53626569728,0,0,0,0,0,0,0,0,0,0,0,0,0,0,33.999998,Fear
155,2018-02-19,9664.730469,55388419584,0,0,0,0,0,0,0,0,0,0,0,0,0,0,63.385712,Greed
156,2018-02-26,11512.599609,48902649856,1,0,0,0,0,0,0,0,0,0,0,0,0,0,44.528570,Neutral
157,2018-03-05,9578.629883,49671590400,0,0,1,0,0,0,0,0,0,0,0,0,0,0,46.571429,Neutral
158,2018-03-12,8223.679688,42075919360,0,0,0,0,0,0,0,0,0,0,0,0,0,0,41.385713,Fear
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,2023-05-29,27119.066406,91351657572,0,0,0,0,0,0,0,0,0,0,0,0,0,0,50.499998,Neutral
431,2023-06-05,25940.167969,116498927148,0,0,0,0,0,0,0,0,0,0,0,0,0,0,52.385713,Neutral
432,2023-06-12,26336.212891,92905085602,0,0,0,0,0,0,0,0,0,0,0,0,0,0,48.928568,Neutral
433,2023-06-19,30480.261719,138005623534,1,1,1,1,0,0,1,1,1,1,0,0,0,0,46.099999,Neutral


In [9]:
rf = pd.read_excel('C:/Users/limam/Documents/GitHub/Datathon_2023/Dados/risk_free_all_d.xlsx')
rf['r_free_rate'] = rf['r_free_rate'].ffill()
rf['r_free_w'] = rf['r_free_rate'].rolling(7).mean()
rf = rf[['observation_date', 'r_free_w']]
rf['observation_date'] = pd.to_datetime(rf['observation_date'])
rf.dropna(inplace=True)
rf


#rf = pd.read_excel('C:/Users/limam/Documents/GitHub/Datathon_2023/Dados/risk_free_d.xlsx')
#rf['r_free_day'] = rf['r_free_day'].ffill()
#rf['r_free_w'] = rf['r_free_day'].rolling(7).mean()
#rf = rf[['observation_date', 'r_free_w']]
#rf['observation_date'] = pd.to_datetime(rf['observation_date'])
#rf.head(10)

,observation_date,r_free_w
6,2018-01-10,0.000051
7,2018-01-11,0.000051
8,2018-01-12,0.000051
9,2018-01-15,0.000051
10,2018-01-16,0.000052
...,...,...
1531,2023-11-15,0.000213
1532,2023-11-16,0.000213
1533,2023-11-17,0.000213
1534,2023-11-20,0.000213


In [10]:
dummies = pd.get_dummies(df['sentiment'])
df = pd.concat([df.drop('sentiment', axis=1), dummies], axis=1)
df['Returns'] = df['Adj Close'].pct_change()
df = df.merge(rf, how='left', left_on='Date', right_on='observation_date').dropna()
df.drop('observation_date', axis=1, inplace=True)
df['risk_premium'] = df['Returns'] - df['r_free_w']
df.head(10)

,Date,Adj Close,Volume,MA_1_9,MA_1_12,MA_2_9,MA_2_12,MA_3_9,MA_3_12,MOM_9,...,VOL_3_12,weekly_mean,Extreme Fear,Extreme Greed,Fear,Greed,Neutral,Returns,r_free_w,risk_premium
1,2018-02-19,9664.730469,55388419584,0,0,0,0,0,0,0,...,0,63.385712,False,False,False,True,False,-0.084068,0.000052,-0.084121
2,2018-02-26,11512.599609,48902649856,1,0,0,0,0,0,0,...,0,44.528570,False,False,False,False,True,0.191197,0.000054,0.191143
3,2018-03-05,9578.629883,49671590400,0,0,1,0,0,0,0,...,0,46.571429,False,False,False,False,True,-0.167987,0.000058,-0.168045
4,2018-03-12,8223.679688,42075919360,0,0,0,0,0,0,0,...,0,41.385713,False,False,True,False,False,-0.141456,0.000061,-0.141517
5,2018-03-19,8495.780273,40853020160,0,0,0,0,0,0,0,...,0,32.428569,False,False,True,False,False,0.033087,0.000065,0.033022
6,2018-03-26,6844.229980,37970689536,0,0,0,0,0,0,0,...,0,31.642855,False,False,True,False,False,-0.194397,0.000067,-0.194464
7,2018-04-02,7023.520020,31804380416,0,0,0,0,0,0,0,...,0,14.514284,True,False,False,False,False,0.026196,0.000066,0.026130
8,2018-04-09,8329.110352,40915320320,0,0,0,0,0,0,0,...,0,17.185713,True,False,False,False,False,0.185888,0.000066,0.185823
9,2018-04-23,9419.080078,61882420224,1,1,1,1,1,0,0,...,1,25.671427,False,False,True,False,False,0.130863,0.000065,0.130797
10,2018-04-30,9654.799805,57244449280,1,1,1,1,1,1,1,...,1,53.485712,False,False,False,False,True,0.025026,0.000065,0.024961


In [11]:
r2 = {}

for i in range(3, 17):
    y = df['risk_premium'][1:df.shape[0]].values
    x = df.iloc[0:df.shape[0]-1, i].values
    x = sm.add_constant(x)  
    model = sm.OLS(y, x).fit()
    e = model.resid
    Ic = []
    for j in range(18, 23):
        r_squared = 1 - (np.sum(df.iloc[1:df.shape[0], j] * e**2) / np.sum(df.iloc[1:df.shape[0], j] * ((y - np.mean(y))**2)))
        #print(f"Out-of-Sample R² for technical indicator {df.columns[i]} - {df.columns[j]}: {r_squared}")

        Ic.append(r_squared)
    r2[df.columns[i]] = Ic

In [16]:
len(x)

237

In [12]:
r2_df = pd.DataFrame(r2)
r2_df = r2_df.T 
r2_df.columns = ['Extreme Fear', 'Extreme Greed', 'Fear', 'Greed', 'Neutral']
r2_df

,Extreme Fear,Extreme Greed,Fear,Greed,Neutral
MA_1_9,-0.019052,0.024990,0.023403,0.008752,-0.011526
MA_1_12,-0.060774,0.026324,0.035407,0.008381,0.014101
MA_2_9,-0.019932,0.017783,0.006211,0.006763,0.007500
MA_2_12,-0.036984,0.020913,0.018649,0.006794,0.005195
MA_3_9,-0.021886,0.023263,0.004299,0.029758,0.016514
MA_3_12,-0.026322,0.017183,0.002591,0.022453,0.010260
MOM_9,-0.074345,0.039924,0.034686,0.019103,0.024069
MOM_12,-0.051689,0.022205,0.021129,0.019460,0.013534
VOL_1_9,-0.022849,0.036659,0.014197,0.006067,0.016768
VOL_1_12,-0.021096,0.017488,0.009851,0.003265,0.009402
